__Deep learning with PyTorch: A 60 minute blitz__

1. [Load tools](#Load-tools)
1. [What is PyTorch?](#What-is-PyTorch?)
    1. [Tensors](#Tensors)
    1. [Operations](#Operations)
    1. [Numpy bridge](#Numpy-bridge)    
    1. [CUDA tensors](#CUDA-tensors)
1. [Autograd - automatic differentiation](#Autograd-automatic-differentiation)
    1. [](#)
    1. [](#)
1. [Neural networks](#Neural-networks)
    1. [](#)
    1. [](#)
1. [Training a classifier](#Training-a-classifier)
    1. [](#)
    1. [](#)
1. [Data parallelism](#Data-parallelism)
    1. [](#)
    1. [](#)

# Load tools

<a id = 'Load-tools'></a>

In [2]:
# Standard libary and settings
import os
import sys
import warnings; warnings.simplefilter('ignore')
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))

# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.6f}'.format

# import PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
import torch. autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.jit import script, trace

# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt

# Magic functions
%matplotlib inline


# What is PyTorch?

<a id = 'What-is-PyTorch?'></a>

## Tensors

<a id = 'Tensors'></a>

In [3]:
# uninitialized 3 by 3 matrix
x = torch.empty(5, 3)
x 


tensor([[-9.7300e+11,  4.5652e-41,  3.6386e+24],
        [ 3.0714e-41,  1.3563e-19,  1.3563e-19],
        [ 1.3563e-19,  9.8439e-12,  7.1390e+31],
        [ 1.8037e+28,  2.0706e-19,  1.9349e-19],
        [ 3.0263e+29,  1.1728e-19,  2.7487e+20]])

In [6]:
# randomly initialized matrix
x = torch.rand(5, 3)
x


tensor([[0.8276, 0.7862, 0.5358],
        [0.2691, 0.0500, 0.1871],
        [0.4402, 0.6702, 0.9650],
        [0.5082, 0.3722, 0.9869],
        [0.5659, 0.7176, 0.0205]])

In [7]:
# initialize matrix filled with zeros and of data type long
x = torch.zeros(5, 3, dtype = torch.long)
x


tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [11]:
# construct tensor directly from input data
x = torch.tensor([5.5, 3])
x


tensor([5.5000, 3.0000])

In [12]:
# construct tensor based on existing tensor
# reuses properties of input tensor such as datatype, unless overridden
x = x.new_ones(5, 5, dtype = torch.double)
x


tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float64)

In [13]:
#
x = torch.randn_like(x, dtype = torch.float)
x


tensor([[ 1.0522, -1.0460,  0.4419, -0.1849, -0.1907],
        [ 1.6835,  1.6386, -0.8171, -1.1147, -0.0928],
        [-0.9165, -1.4796, -0.7995,  0.9421, -1.2656],
        [-1.2507, -1.1456, -0.0069,  1.6657,  0.5465],
        [ 0.4836,  0.9832,  0.3220,  0.8043,  0.7400]])

In [14]:
#
torch.Size([5, 3])

torch.Size([5, 3])

## Operations

<a id = 'Operations'></a>

In [16]:
# add two tensors
x = torch.rand(5, 3)
y = torch.rand(5, 3)
print(x + y)


tensor([[1.1701, 0.9582, 1.7936],
        [0.7377, 0.3268, 0.3625],
        [1.3486, 1.0652, 1.0988],
        [1.4752, 1.4661, 0.5582],
        [0.9773, 1.0988, 0.9155]])


In [17]:
# alternative addition syntax
print(torch.add(x, y))


tensor([[1.1701, 0.9582, 1.7936],
        [0.7377, 0.3268, 0.3625],
        [1.3486, 1.0652, 1.0988],
        [1.4752, 1.4661, 0.5582],
        [0.9773, 1.0988, 0.9155]])


In [19]:
# provide an output tensor as an argument
result = torch.empty(5, 3)
torch.add(x, y, out = result)
result


tensor([[1.1701, 0.9582, 1.7936],
        [0.7377, 0.3268, 0.3625],
        [1.3486, 1.0652, 1.0988],
        [1.4752, 1.4661, 0.5582],
        [0.9773, 1.0988, 0.9155]])

In [20]:
# in-place addition. in-place operators have an underscore suffix
y.add_(x)
y


tensor([[1.1701, 0.9582, 1.7936],
        [0.7377, 0.3268, 0.3625],
        [1.3486, 1.0652, 1.0988],
        [1.4752, 1.4661, 0.5582],
        [0.9773, 1.0988, 0.9155]])

In [23]:
# numpy-esque slicing
y[:, 1]


tensor([0.9582, 0.3268, 1.0652, 1.4661, 1.0988])

In [25]:
# resize/reshape
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8) # -1 is inferred from other dimensions

print('4 by 4\n')
print(x)
print('\n1 by 16\n')
print(y)
print('\n2 by 8\n')
print(z)


4 by 4

tensor([[-0.1141,  0.5764, -0.1237, -0.7859],
        [-1.8387,  0.4665, -0.6362, -0.1919],
        [ 0.0670,  0.1599, -0.2152, -0.1759],
        [ 0.7113, -1.2659, -0.7085, -0.5246]])

1 by 16

tensor([-0.1141,  0.5764, -0.1237, -0.7859, -1.8387,  0.4665, -0.6362, -0.1919,
         0.0670,  0.1599, -0.2152, -0.1759,  0.7113, -1.2659, -0.7085, -0.5246])

2 by 8

tensor([[-0.1141,  0.5764, -0.1237, -0.7859, -1.8387,  0.4665, -0.6362, -0.1919],
        [ 0.0670,  0.1599, -0.2152, -0.1759,  0.7113, -1.2659, -0.7085, -0.5246]])


In [26]:
# access data point in one element tensor
x = torch.randn(1)
print(x)
print(x.item())


tensor([-0.6776])
-0.6775725483894348


## Numpy bridge

<a id = 'Numpy-bridge'></a>

In [28]:
# convery torch tensor to numpy array
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)


tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [29]:
# value update reflected in both the tensor and array
a.add_(1)
print(a)
print(b)


tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [32]:
# convert numpy array to torch tensor
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out = a)
print(a)
print(b)


[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


## CUDA tensors

<a id = 'CUDA-tensors'></a>

In [ ]:
#


# Autograd - automatic differentiation

<a id = 'Autograd-automatic-differentiation'></a>

# Neural networks

<a id = 'Neural-networks'></a>

# Training a classifier

<a id = 'Training-a-classifier'></a>

# Data parallelism

<a id = 'Data-parallelism'></a>